In [ ]:
!pip install stable_baselines3
!pip install ortools
!pip install shimmy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.2

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import gym
from gym import spaces

Parsing script for Solomon data and enviornment

In [ ]:

def parse_solomon_dataset(file_path):
    """
    Parse Solomon VRPTW C101 dataset
    Returns: Dictionary with problem instance data
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse header information
    vehicle_capacity = int(lines[4].strip().split()[1])

    # Parse customer data (including depot as first customer)
    customers = []
    for line in lines[9:]:  # Data starts at line 9
        if line.strip():
            parts = [float(x) for x in line.strip().split()]
            customer = {
                'id': int(parts[0]),
                'x': parts[1],
                'y': parts[2],
                'demand': parts[3],
                'ready_time': parts[4],
                'due_time': parts[5],
                'service_time': parts[6]
            }
            customers.append(customer)

    return {
        'vehicle_capacity': vehicle_capacity,
        'customers': customers
    }

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Models

Models and methods

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import gym
from gym import spaces

# Define VRP Environment
class VRPEnvironment:
    def __init__(self, num_customers=10, grid_size=50, solomon_data=None):
        self.num_customers = num_customers
        self.grid_size = grid_size
        self.travel_time = 0
        self.travel_distance = 0

        if solomon_data:
            # Initialize from Solomon dataset
            self.init_from_solomon(solomon_data)
        else:
            # Random initialization
            self.vehicle_capacity = 200  # Increased for more realistic scenarios
            self.depot = np.array([grid_size/2, grid_size/2])
            self.customers = np.random.randint(0, grid_size, size=(num_customers, 2))
            self.demands = np.random.randint(1, 30, size=num_customers)
            self.ready_times = np.zeros(num_customers)
            self.due_times = np.ones(num_customers) * 1000  # More realistic time windows
            self.service_times = np.ones(num_customers) * 10  # Standard service time

        # Initialize state variables that will be reset
        self.reset()

    def reset(self):
        """Reset the environment to initial state"""
        self.visited = np.zeros(self.num_customers, dtype=bool)
        self.current_pos = self.depot.copy()
        self.current_time = 0
        self.current_load = 0
        self.travel_time = 0
        self.travel_distance = 0
        return self.get_state()

    def init_from_solomon(self, solomon_data):
        """Initialize environment from Solomon dataset with proper scaling"""
        customers = solomon_data['customers']
        self.num_customers = len(customers) - 1  # Exclude depot
        self.vehicle_capacity = solomon_data['vehicle_capacity']

        # Set depot (first customer in Solomon format)
        depot = customers[0]
        self.depot = np.array([depot['x'], depot['y']])

        # Initialize arrays for customers (excluding depot)
        self.customers = np.array([[c['x'], c['y']] for c in customers[1:]])
        self.demands = np.array([c['demand'] for c in customers[1:]])
        self.ready_times = np.array([c['ready_time'] for c in customers[1:]])
        self.due_times = np.array([c['due_time'] for c in customers[1:]])
        self.service_times = np.array([c['service_time'] for c in customers[1:]])


    def get_state(self):
        """Return current state with proper normalization"""
        # Normalize coordinates
        max_coord = max(np.max(self.customers), np.max(self.depot))
        normalized_pos = self.current_pos / max_coord
        normalized_customers = self.customers / max_coord

        # Normalize time windows
        max_time = max(np.max(self.due_times), self.current_time)
        normalized_time = self.current_time / max_time if max_time > 0 else 0

        # Normalize capacity
        normalized_load = self.current_load / self.vehicle_capacity if self.vehicle_capacity > 0 else 0

        return np.concatenate([
            normalized_pos,                    # Current position (2,)
            normalized_customers.flatten(),    # Customer coordinates (num_customers * 2,)
            self.visited.astype(float),       # Visit status (num_customers,)
            [normalized_time],                # Current time (1,)
            [normalized_load],                # Current load (1,)
            self.demands / np.max(self.demands) if np.max(self.demands) > 0 else self.demands,  # Normalized demands
            self.ready_times / max_time if max_time > 0 else self.ready_times,  # Normalized ready times
            self.due_times / max_time if max_time > 0 else self.due_times      # Normalized due times
        ])

    def step(self, action):
        # Check if action is valid
        if action >= self.num_customers:
            return self.get_state(), -100, True

        # Calculate distance to next customer
        next_pos = self.customers[action]
        print("Action:",action)

        print("Current Pos:",self.current_pos)
        print("Next Post", next_pos)
        print(" prev travel dist", self.travel_distance)
        self.travel_distance += np.abs(self.current_pos - next_pos).sum() #manhattan distance for grid
        print("post travel dist", self.travel_distance)
        done = np.all(self.visited)  # Ensure done is defined here

        #print(f"Action mask = {self.action_mask}")
        print(f"Visited = {self.visited}")
        print(f"# cust= {self.num_customers}")

        print(f"Done  = {done}")

        # Update state
        self.current_pos = next_pos
        if self.visited[action]: # already visited - exit
            return self.get_state(), -100, done
        self.visited[action] = True
        num_visited = np.sum(self.visited)

        # Calculate reward components
        progress_reward = 100  # Reward for visiting a new customer

        distance_penalty = -self.travel_distance
        completion_bonus = 1000 if done else 0
        total_reward = distance_penalty + completion_bonus

        print("Total reward", total_reward)

        return self.get_state(), total_reward, done


class GymVRPEnvironment(gym.Env):
    def __init__(self, num_customers=10, grid_size=50, solomon_data=None):
        super(GymVRPEnvironment, self).__init__()
        self.env = VRPEnvironment(num_customers, grid_size, solomon_data)

        # Dynamic state space based on problem size
        state_size = (2 +                     # Current position
                     self.env.num_customers * 2 +  # Customer coordinates
                     self.env.num_customers +      # Visit status
                     1 +                          # Current time
                     1 +                          # Current load
                     self.env.num_customers * 3)  # Demands and time windows

        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(state_size,),
            dtype=np.float32
        )

        self.action_space = spaces.Discrete(self.env.num_customers)

    def reset(self):
        return self.env.reset().astype(np.float32)

    def step(self, action):
        next_state, reward, done = self.env.step(action)
        return next_state.astype(np.float32), reward, done, {}

# Define RL and OR Methods
# OR-Tools Solver
class ORToolsSolver:
    def solve(self, env):
        state = env.reset()
        depot = 0
        num_customers = env.num_customers

        # Create distance matrix
        distance_matrix = np.zeros((num_customers + 1, num_customers + 1))
        all_points = np.vstack([env.depot, env.customers])
        for i in range(num_customers + 1):
            for j in range(num_customers + 1):
                distance_matrix[i][j] = np.abs(all_points[i] - all_points[j]).sum()

        # Set up routing
        manager = pywrapcp.RoutingIndexManager(num_customers + 1, 1, depot)
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return int(distance_matrix[from_node][to_node])

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Solve
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        solution = routing.SolveWithParameters(search_parameters)
        route = []

        if solution:
            index = routing.Start(0)
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index) - 1)  # Adjust for depot
                index = solution.Value(routing.NextVar(index))
            route = [node for node in route if node >= 0]  # Remove depot if included

        return route  # Return the visiting order


# PPO Agent
class PPOAgent:
    def __init__(self, env, policy='MlpPolicy', **kwargs):
        self.env = DummyVecEnv([lambda: env])
        self.model = PPO(policy, self.env, **kwargs)

    def train(self, timesteps):
        self.model.learn(total_timesteps=timesteps)

    def evaluate(self, episodes=10):
        rewards = []
        for _ in range(episodes):
            state = self.env.reset()
            episode_reward = 0
            done = False

            while not done:
                action, _ = self.model.predict(state)
                state, reward, done, _ = self.env.step(action)
                episode_reward += reward[0]

            rewards.append(episode_reward)

        return rewards

    def select_action(self, state):
        """Modified to handle numpy array actions"""
        action, _ = self.model.predict(state)
        return int(action[0]) if isinstance(action, np.ndarray) else int(action)

# Baseline DQN Implementation
class DQNNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):  # Increased hidden dim
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),  # Added another layer
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

    def forward(self, x):
        return self.net(x)

class DQNAgent:
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=1e-3, gamma=0.95):
        self.device = torch.device("cpu")
        self.gamma = gamma
        self.action_dim = action_dim

        self.network = DQNNetwork(state_dim, action_dim, hidden_dim).to(self.device)
        self.target_network = DQNNetwork(state_dim, action_dim, hidden_dim).to(self.device)
        self.target_network.load_state_dict(self.network.state_dict())

        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)
        self.memory = deque(maxlen=10000)
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)

        with torch.no_grad():
            state = torch.FloatTensor(state).to(self.device)
            q_values = self.network(state)
            return q_values.argmax().item()

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def update(self, batch_size=32):
        if len(self.memory) < batch_size:
            return

        batch = random.sample(self.memory, batch_size)
        state_batch = torch.FloatTensor([t[0] for t in batch]).to(self.device)
        action_batch = torch.LongTensor([t[1] for t in batch]).to(self.device)
        reward_batch = torch.FloatTensor([t[2] for t in batch]).to(self.device)
        next_state_batch = torch.FloatTensor([t[3] for t in batch]).to(self.device)
        done_batch = torch.FloatTensor([t[4] for t in batch]).to(self.device)

        current_q = self.network(state_batch).gather(1, action_batch.unsqueeze(1))
        next_q = self.target_network(next_state_batch).max(1)[0].detach()
        target_q = reward_batch + (1 - done_batch) * self.gamma * next_q

        loss = F.mse_loss(current_q.squeeze(), target_q)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        return loss.item()

class MaxEntNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super().__init__()

        self.policy_net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

        self.value_net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        self.action_dim = action_dim


    def policy(self, state):
        logits = self.policy_net(state)
        dist = F.softmax(logits, dim=-1)
        log_prob = F.log_softmax(logits, dim=-1)
        entropy = -(dist * log_prob).sum(dim=-1)
        return dist, entropy

    def value(self, state):
        return self.value_net(state)

class MaxEntRL:
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=1e-3, gamma=0.95, tau=0.1, alpha=0.1):
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha
        self.action_dim = action_dim

        # Force CPU usage
        self.device = torch.device("cpu")

        self.network = MaxEntNetwork(state_dim, action_dim, hidden_dim).to(self.device)
        self.target_network = MaxEntNetwork(state_dim, action_dim, hidden_dim).to(self.device)
        self.target_network.load_state_dict(self.network.state_dict())

        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)

        self.memory = deque(maxlen=10000)  # Smaller memory

    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(self.device)
            dist, _ = self.network.policy(state)
            # Mask invalid actions
            #mask = state[-self.action_dim:].bool()
            # Mask indice was fixed - once visited it will be set to false
            mask = ~state[self.action_dim*2+2:self.action_dim*3+2].bool()

            dist = dist * mask
            dist = dist / (dist.sum() + 1e-10)  # Renormalize
            if dist.sum() == 0: # if all the customers are visited and mask is all false
                dist[0] = 1 # send to depot
            action = torch.multinomial(dist, 1).item()
        return action
    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def update(self, batch_size=32):  # Smaller batch size
        if len(self.memory) < batch_size:
            return

        batch = random.sample(self.memory, batch_size)
        state_batch = torch.FloatTensor([t[0] for t in batch]).to(self.device)
        action_batch = torch.LongTensor([t[1] for t in batch]).to(self.device)
        reward_batch = torch.FloatTensor([t[2] for t in batch]).to(self.device)
        next_state_batch = torch.FloatTensor([t[3] for t in batch]).to(self.device)
        done_batch = torch.FloatTensor([t[4] for t in batch]).to(self.device)

        with torch.no_grad():
            next_value = self.target_network.value(next_state_batch).squeeze()
            expected_value = reward_batch + self.gamma * next_value * (1 - done_batch)

        curr_value = self.network.value(state_batch).squeeze()
        value_loss = F.mse_loss(curr_value, expected_value)

        dist, entropy = self.network.policy(state_batch)
        log_prob = torch.log(dist + 1e-10)
        advantage = (expected_value - curr_value).detach()

        policy_loss = -(log_prob[range(batch_size), action_batch] * advantage).mean()
        entropy_loss = -entropy.mean()

        total_loss = value_loss + policy_loss + self.alpha * entropy_loss

        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

        # Soft update target network
        for target_param, param in zip(self.target_network.parameters(), self.network.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        return total_loss.item()

# SAC Implementation (simplified version)
class SACNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super().__init__()

        # Actor network outputs mean and log_std for each action
        self.actor = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim * 2)  # Mean and log_std for each action
        )

        # Two Q-networks for double Q-learning
        self.q1 = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)  # Q-value for each action
        )

        self.q2 = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)  # Q-value for each action
        )

        # Target Q-networks
        self.target_q1 = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

        self.target_q2 = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

class SACAgent:
    def __init__(self, state_dim, action_dim, hidden_dim=256, lr=3e-4, gamma=0.99, tau=0.005, alpha=0.2, buffer_size=100000):
        self.device = torch.device("cpu")
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha
        self.action_dim = action_dim

        self.network = SACNetwork(state_dim, action_dim, hidden_dim).to(self.device)

        # Separate optimizers for actor and critics
        self.actor_optimizer = optim.Adam(self.network.actor.parameters(), lr=lr)
        self.q1_optimizer = optim.Adam(self.network.q1.parameters(), lr=lr)
        self.q2_optimizer = optim.Adam(self.network.q2.parameters(), lr=lr)

        # Initialize target networks
        for target_param, param in zip(self.network.target_q1.parameters(), self.network.q1.parameters()):
            target_param.data.copy_(param.data)
        for target_param, param in zip(self.network.target_q2.parameters(), self.network.q2.parameters()):
            target_param.data.copy_(param.data)

        self.memory = deque(maxlen=buffer_size)

        # Temperature parameter
        self.target_entropy = -action_dim  # Target entropy is -|A|
        self.log_alpha = torch.zeros(1, requires_grad=True, device=self.device)
        self.alpha_optimizer = optim.Adam([self.log_alpha], lr=lr)

    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(self.device)
            output = self.network.actor(state)
            mean, log_std = output.chunk(2, dim=-1)
            log_std = torch.clamp(log_std, -20, 2)
            std = log_std.exp()

            # Use reparameterization trick
            normal = torch.distributions.Normal(mean, std)
            action = normal.rsample()

            # Apply softmax and mask
            action_probs = F.softmax(action, dim=-1)
            #mask = state[-self.action_dim:].bool()
            # Mask indice was fixed - once visited it will be set to false
            mask = ~state[self.action_dim*2+2:self.action_dim*3+2].bool()

            action_probs = action_probs * mask
            action_probs = action_probs / (action_probs.sum() + 1e-10)
            return action_probs.argmax().item()

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def update(self, batch_size=256):
        if len(self.memory) < batch_size:
            return

        # Sample batch
        batch = random.sample(self.memory, batch_size)
        state_batch = torch.FloatTensor([t[0] for t in batch]).to(self.device)
        action_batch = torch.LongTensor([t[1] for t in batch]).to(self.device)
        reward_batch = torch.FloatTensor([t[2] for t in batch]).to(self.device)
        next_state_batch = torch.FloatTensor([t[3] for t in batch]).to(self.device)
        done_batch = torch.FloatTensor([t[4] for t in batch]).to(self.device)

        # Update temperature parameter
        actor_output = self.network.actor(state_batch)
        mean, log_std = actor_output.chunk(2, dim=-1)
        log_std = torch.clamp(log_std, -20, 2)
        std = log_std.exp()
        normal = torch.distributions.Normal(mean, std)
        sampled_actions = normal.rsample()
        log_probs = normal.log_prob(sampled_actions).sum(dim=-1)

        alpha_loss = -(self.log_alpha * (log_probs + self.target_entropy).detach()).mean()
        self.alpha_optimizer.zero_grad()
        alpha_loss.backward()
        self.alpha_optimizer.step()

        self.alpha = self.log_alpha.exp().item()

        # Update critics
        with torch.no_grad():
            next_actor_output = self.network.actor(next_state_batch)
            next_mean, next_log_std = next_actor_output.chunk(2, dim=-1)
            next_log_std = torch.clamp(next_log_std, -20, 2)
            next_std = next_log_std.exp()
            next_normal = torch.distributions.Normal(next_mean, next_std)
            next_actions = next_normal.rsample()
            next_log_probs = next_normal.log_prob(next_actions).sum(dim=-1)

            next_action_probs = F.softmax(next_actions, dim=-1)
            next_q1 = self.network.target_q1(next_state_batch)
            next_q2 = self.network.target_q2(next_state_batch)
            next_q = torch.min(next_q1, next_q2)
            next_q = (next_action_probs * next_q).sum(dim=1)
            target_q = reward_batch + (1 - done_batch) * self.gamma * (next_q - self.alpha * next_log_probs)

        # Get current Q estimates
        current_q1 = self.network.q1(state_batch)
        current_q2 = self.network.q2(state_batch)
        current_q1 = current_q1.gather(1, action_batch.unsqueeze(1)).squeeze()
        current_q2 = current_q2.gather(1, action_batch.unsqueeze(1)).squeeze()

        # Compute critic losses
        q1_loss = F.mse_loss(current_q1, target_q)
        q2_loss = F.mse_loss(current_q2, target_q)

        # Update critics
        self.q1_optimizer.zero_grad()
        q1_loss.backward()
        self.q1_optimizer.step()

        self.q2_optimizer.zero_grad()
        q2_loss.backward()
        self.q2_optimizer.step()

        # Update actor
        actor_output = self.network.actor(state_batch)
        mean, log_std = actor_output.chunk(2, dim=-1)
        log_std = torch.clamp(log_std, -20, 2)
        std = log_std.exp()
        normal = torch.distributions.Normal(mean, std)
        actions = normal.rsample()
        log_probs = normal.log_prob(actions).sum(dim=-1)

        action_probs = F.softmax(actions, dim=-1)
        q1 = self.network.q1(state_batch)
        q2 = self.network.q2(state_batch)
        q = torch.min(q1, q2)
        q = (action_probs * q).sum(dim=1)

        actor_loss = (self.alpha * log_probs - q).mean()

        # Update actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Soft update target networks
        for target_param, param in zip(self.network.target_q1.parameters(), self.network.q1.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        for target_param, param in zip(self.network.target_q2.parameters(), self.network.q2.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        return actor_loss.item()

class NearestNeighbor:
    def solve(self, env):
        state = env.reset()
        current_pos = env.depot.copy()
        unvisited = set(range(env.num_customers))
        route = []  # To track the visiting order
        total_distance = 0

        while unvisited:
            nearest =  min(unvisited, key=lambda i: np.abs(current_pos - env.customers[i]).sum())
            total_distance += np.abs(current_pos - env.customers[nearest]).sum()
            route.append(nearest)
            current_pos = env.customers[nearest]
            unvisited.remove(nearest)

        total_distance += np.abs(current_pos - env.depot).sum()
        route.append(0)  # Return to depot
        return route  # Return the visiting order




In [ ]:
def calculate_route_distance(env, route):
    """
    Calculate the total route distance:
    1. Start from depot
    2. Visit each customer in route order
    3. Return to depot
    """
    if not route or len(set(route)) < env.num_customers:  # Check if route is complete
        return float('inf')  # Invalid/incomplete route

    total_distance = 0
    current_pos = env.depot.copy()

    # Follow route customer by customer
    for customer_idx in route:
        if customer_idx >= env.num_customers:  # Invalid customer index
            return float('inf')

        # Distance to next customer
        customer_pos = env.customers[customer_idx]
        distance = np.abs(current_pos - customer_pos).sum()
        total_distance += distance
        current_pos = customer_pos.copy()

    # Return to depot
    total_distance += np.abs(current_pos - env.depot).sum()

    return total_distance

def validate_route(env, route):
    """
    Validate if route is complete and feasible:
    1. All customers visited exactly once
    2. Valid customer indices
    3. Capacity constraints respected
    4. Time windows respected
    """
    env = env.env
    if not route:
        return False, "Empty route"

    # Check if all customers are visited exactly once
    visited = set(route)
    if len(visited) != env.num_customers:
        return False, f"Not all customers visited. Visited {len(visited)}/{env.num_customers}"

    # Check valid indices
    if any(i >= env.num_customers for i in route):
        return False, "Invalid customer index"

    # Check capacity constraints
    total_load = 0
    for customer_idx in route:
        total_load += env.demands[customer_idx]
        if total_load > env.vehicle_capacity:
            return False, "Capacity constraint violated"

    # Check time windows
    current_time = 0
    current_pos = env.depot.copy()

    for customer_idx in route:
        customer_pos = env.customers[customer_idx]
        travel_time = np.abs(current_pos - customer_pos).sum()

        arrival_time = current_time + travel_time
        if arrival_time > env.due_times[customer_idx]:
            return False, "Time window violated - arrived too late"

        service_start = max(arrival_time, env.ready_times[customer_idx])
        current_time = service_start + env.service_times[customer_idx]
        current_pos = customer_pos.copy()

    # Check return to depot
    final_time = current_time + np.abs(current_pos - env.depot).sum()
    if final_time > max(env.due_times):
        return False, "Cannot return to depot within time window"

    return True, "Valid route"

class RouteTracker:
    def __init__(self):
        self.episode_routes = []
        self.episode_distances = []
        self.episode_rewards = []
        self.best_route = None
        self.best_distance = float('inf')

    def add_episode(self, env, route, reward):

        # Calculate distance
        distance = calculate_route_distance(env.env, route)
        valid, msg = validate_route(env, route)
        print(f"adding episode =  {route}, valid = {valid}, reward = {reward}")
        if valid and distance < self.best_distance:
            self.best_distance = distance
            self.best_route = route.copy()
            print(f"New best route found! Distance: {distance:.2f}")
            print(f"Route: {route}")

        self.episode_routes.append(route)
        self.episode_distances.append(distance)
        self.episode_rewards.append(reward)

    def get_statistics(self):
        valid_distances = [d for d in self.episode_distances if d != float('inf')]
        avg_distance = np.mean(valid_distances) if valid_distances else float('inf')

        return {
            'avg_distance': avg_distance,
            'best_distance': self.best_distance,
            'best_route': self.best_route,
            'avg_reward': np.mean(self.episode_rewards),
            'completion_rate': len(valid_distances) / len(self.episode_distances),
            'all_distances': self.episode_distances,
            'all_rewards': self.episode_rewards
        }

def print_route_details(env, route):
    """Print detailed information about a route"""
    if not route:
        print("Empty route!")
        return

    print("\nRoute Details:")
    print(f"Number of customers visited: {len(set(route))}/{env.num_customers}")

    total_distance = 0
    current_time = 0
    current_pos = env.depot.copy()
    total_load = 0

    print("\nStep-by-step route:")
    print(f"Start at depot: {env.depot}")

    for i, customer_idx in enumerate(route, 1):
        customer_pos = env.customers[customer_idx]
        distance = np.abs(current_pos - customer_pos).sum()
        total_distance += distance

        arrival_time = current_time + distance
        service_start = max(arrival_time, env.ready_times[customer_idx])
        total_load += env.demands[customer_idx]

        print(f"\nStep {i}:")
        print(f"Visit customer {customer_idx} at position {customer_pos}")
        print(f"Travel distance: {distance:.2f}")
        print(f"Arrival time: {arrival_time:.2f}")
        print(f"Service start: {service_start:.2f}")
        print(f"Due time: {env.due_times[customer_idx]:.2f}")
        print(f"Current load: {total_load}/{env.vehicle_capacity}")

        current_time = service_start + env.service_times[customer_idx]
        current_pos = customer_pos.copy()

    # Return to depot
    final_distance = np.abs(current_pos - env.depot).sum()
    total_distance += final_distance
    final_time = current_time + final_distance

    print("\nReturn to depot:")
    print(f"Final leg distance: {final_distance:.2f}")
    print(f"Final arrival time: {final_time:.2f}")
    print(f"\nTotal route distance: {total_distance:.2f}")
    print(f"Total time: {final_time:.2f}")
    print(f"Total load: {total_load}/{env.vehicle_capacity}")

In [ ]:
def train_agent(env, agent, num_episodes=10):
    tracker = RouteTracker()

    for episode in range(num_episodes):
        state = env.reset()
        current_route = []
        episode_reward = 0
        done = False

        while not done:
            action = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            # Store transition for learning
            agent.store_transition(state, action, reward, next_state, done)
            agent.update()

            # Track route
            current_route.append(action)
            episode_reward += reward
            state = next_state

        # Add episode to tracker
        tracker.add_episode(env, current_route, episode_reward)

        # Print progress every 100 episodes
        if (episode + 1) % 100 == 0:
            stats = tracker.get_statistics()
            print(f"\nEpisode {episode + 1}")
            print(f"Average Distance: {stats['avg_distance']:.2f}")
            print(f"Best Distance: {stats['best_distance']:.2f}")
            print(f"Average Reward: {stats['avg_reward']:.2f}")
            print(f"Completion Rate: {stats['completion_rate']*100:.1f}%")

    return tracker

# Usage example:
env = GymVRPEnvironment(num_customers=10)
#agent = DQNAgent(state_dim=env.observation_space.shape[0],action_dim=env.action_space.n,hidden_dim=256)
agent = SACAgent(state_dim=env.observation_space.shape[0],
                 action_dim=env.action_space.n,
                 hidden_dim=256)

# Train agent
tracker = train_agent(env, agent)

# Print final statistics
stats = tracker.get_statistics()
print("\nFinal Statistics:")
print(f"Best Distance Found: {stats['best_distance']:.2f}")
print(f"Average Distance: {stats['avg_distance']:.2f}")
print(f"Completion Rate: {stats['completion_rate']*100:.1f}%")

# Print detailed analysis of best route
if tracker.best_route is not None:
    print("\nBest Route Details:")
    print_route_details(env.env, tracker.best_route)

Action: 5
Current Pos: [25. 25.]
Next Post [18 34]
 prev travel dist 0
post travel dist 16.0
Visited = [False False False False False False False False False False]
# cust= 10
Done  = False
Total reward -16.0
Action: 0
Current Pos: [18 34]
Next Post [27  8]
 prev travel dist 16.0
post travel dist 51.0
Visited = [False False False False False  True False False False False]
# cust= 10
Done  = False
Total reward -51.0
Action: 7
Current Pos: [27  8]
Next Post [25 43]
 prev travel dist 51.0
post travel dist 88.0
Visited = [ True False False False False  True False False False False]
# cust= 10
Done  = False
Total reward -88.0
Action: 6
Current Pos: [25 43]
Next Post [14 45]
 prev travel dist 88.0
post travel dist 101.0
Visited = [ True False False False False  True False  True False False]
# cust= 10
Done  = False
Total reward -101.0
Action: 8
Current Pos: [14 45]
Next Post [23 22]
 prev travel dist 101.0
post travel dist 133.0
Visited = [ True False False False False  True  True  True Fals

In [ ]:
class RLExperimentRunner:
    def __init__(self, env, sac_agent, maxent_agent, other_agents=None):
        self.env = env
        self.sac_agent = sac_agent
        self.maxent_agent = maxent_agent
        self.other_agents = other_agents or {}
        self.trackers = {
            'SAC': RouteTracker(),
            'MaxEnt': RouteTracker(),
        }
        for name in self.other_agents.keys():
            self.trackers[name] = RouteTracker()

    def run(self, episodes=100):
        for episode in range(episodes):
            print(f"Running Episode {episode + 1}/{episodes}")
            # Run SAC Agent
            self.run_agent(self.sac_agent, self.trackers['SAC'], "SAC")
            # Run MaxEnt Agent
            self.run_agent(self.maxent_agent, self.trackers['MaxEnt'], "MaxEnt")
            # Run Other Agents
            for name, agent in self.other_agents.items():
                self.run_agent(agent, self.trackers[name], name)

    def run_agent(self, agent, tracker, name):
        state = self.env.reset()
        route = []
        done = False
        episode_reward = 0

        while not done:
            action = agent.select_action(state)
            state, reward, done, _ = self.env.step(action)
            episode_reward += reward
            route.append(action)

        valid, msg = validate_route(self.env, route)
        if valid:
            tracker.add_episode(self.env, route, episode_reward)
        else:
            print(f"{name} Agent - Invalid Route: {msg}")

    def summarize_results(self):
        results = {}
        for name, tracker in self.trackers.items():
            print(f"\n{name} Results:")
            print(f"\n{tracker.episode_distances} episode:")
            stats = tracker.get_statistics()
            print(f"Average Distance: {stats['avg_distance']:.2f}")
            print(f"Best Distance: {stats['best_distance']:.2f}")
            print(f"Completion Rate: {stats['completion_rate'] * 100:.2f}%")
            print(f"Average Reward: {stats['avg_reward']:.2f}")
            results[name] = stats
        return results
def validate_route(env, route):
    """
    Validate if route is complete and feasible:
    1. All customers visited exactly once
    2. Valid customer indices
    3. Capacity constraints respected
    4. Time windows respected
    """
    # Access the wrapped environment
    vrp_env = env.env

    if not route:
        return False, "Empty route"

    # Check if all customers are visited exactly once
    visited = set(route)
    if len(visited) != vrp_env.num_customers:
        return False, f"Not all customers visited. Visited {len(visited)}/{vrp_env.num_customers}"

    # Check valid indices
    if any(i >= vrp_env.num_customers for i in route):
        return False, "Invalid customer index"

    # Check capacity constraints
    total_load = 0
    for customer_idx in route:
        total_load += vrp_env.demands[customer_idx]
        if total_load > vrp_env.vehicle_capacity:
            return False, "Capacity constraint violated"

    # Check time windows
    current_time = 0
    current_pos = vrp_env.depot.copy()

    for customer_idx in route:
        customer_pos = vrp_env.customers[customer_idx]
        travel_time = np.abs(current_pos - customer_pos).sum()

        arrival_time = current_time + travel_time
        if arrival_time > vrp_env.due_times[customer_idx]:
            return False, "Time window violated - arrived too late"

        service_start = max(arrival_time, vrp_env.ready_times[customer_idx])
        current_time = service_start + vrp_env.service_times[customer_idx]
        current_pos = customer_pos.copy()

    # Check return to depot
    final_time = current_time + np.abs(current_pos - vrp_env.depot).sum()
    if final_time > max(vrp_env.due_times):
        return False, "Cannot return to depot within time window"

    return True, "Valid route"

env = GymVRPEnvironment(num_customers=29, grid_size=50)
sac_agent = SACAgent(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.n )
maxent_agent = MaxEntRL(
     state_dim=env.observation_space.shape[0],
     action_dim=env.action_space.n
 )
other_agents = {
     #"NearestNeighbor": NearestNeighbor(),
     #"PPO": PPOAgent(env),
}
runner = RLExperimentRunner(env, sac_agent, maxent_agent, other_agents)
runner.run(episodes=50)
results = runner.summarize_results()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Visited = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True False  True  True  True
 False  True  True  True  True]
# cust= 29
Done  = False
Total reward -952.0
Action: 20
Current Pos: [40 20]
Next Post [7 0]
 prev travel dist 952.0
post travel dist 1005.0
Visited = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
 False  True  True  True  True]
# cust= 29
Done  = False
Total reward -1005.0
Action: 24
Current Pos: [7 0]
Next Post [ 5 30]
 prev travel dist 1005.0
post travel dist 1037.0
Visited = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True]
# cust= 29
Done  = False
Total reward -1037.0
Action: 0
Current Pos: [ 5 30]
Next Post [19 15]
 prev 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: division by zero

Experiment on complex enviornment

In [ ]:
# Experiment Setup
def run_experiments(episodes=100, env_size=10):
    # Initialize environment
    env = GymVRPEnvironment(num_customers=env_size)

    # Get actual state dimension from environment
    state = env.reset()
    state_dim = len(state)
    action_dim = env.action_space.n

    print(f"Environment initialized with state_dim={state_dim}, action_dim={action_dim}")

    # Initialize agents with correct dimensions
    maxent_agent = MaxEntRL(state_dim, action_dim, hidden_dim=256)
    sac_agent = SACAgent(state_dim, action_dim, hidden_dim=256)
    # dqn_agent = DQNAgent(state_dim, action_dim, hidden_dim=256)
    # nn_solver = NearestNeighbor()
    # ppo_env = GymVRPEnvironment(num_customers=env_size)
    # ppo_agent = PPOAgent(
    #     ppo_env,
    #     verbose=0,
    #     policy_kwargs={'net_arch': [256, 256]}
    # )
    # or_solver = ORToolsSolver()

    # # History tracking
    # or_rewards = []
    # ppo_rewards = []
    maxent_rewards = []
    sac_rewards = []
    # dqn_rewards = []
    # nn_rewards = []

    # Train PPO Agent
    print("\nTraining PPO...")
    # ppo_agent.model.learn(total_timesteps=episodes * 100)
    # ppo_agent.train(timesteps=episodes * 100)
    # Training loop
    for algorithm, agent, rewards in [
        ("MaxEnt", maxent_agent, maxent_rewards),
        ("SAC", sac_agent, sac_rewards),
        # ("DQN", dqn_agent, dqn_rewards),
    ]:
        print(f"\nTraining {algorithm}...")

        for episode in tqdm(range(episodes)):
            state = env.reset()
            episode_reward = 0
            steps = 0
            done = False

            while not done and steps < env_size * 2:
                steps += 1
                action = agent.select_action(state)
                next_state, reward, done, _ = env.step(action)

                agent.store_transition(state, action, reward, next_state, done)
                agent.update()

                episode_reward += reward
                state = next_state

            rewards.append(episode_reward)

            if (episode + 1) % 20 == 0:
                mean_reward = np.mean(rewards[-20:])
                print(f"\n{algorithm} Episode {episode+1}, Average Reward: {mean_reward:.2f}")

    # Run Nearest Neighbor
    # print("\nRunning Nearest Neighbor...")
    # for _ in tqdm(range(episodes)):
    #     reward = nn_solver.solve(env.env)
    #     nn_rewards.append(reward)

    # Run OR-Tools Solver
    # print("\nRunning OR-Tools Solver...")
    # for _ in tqdm(range(episodes)):
    #     reward = or_solver.solve(env.env)
    #     or_rewards.append(reward)

    # Evaluate PPO
    print("\nEvaluating PPO...")
    # ppo_rewards = ppo_agent.evaluate(episodes=episodes)

    return maxent_rewards, sac_rewards, maxent_agent, sac_agent
    # dqn_rewards, nn_rewards, or_rewards, ppo_rewards, maxent_rewards, sac_rewards, dqn_agent, maxent_agent, sac_agent, ppo_agent, nn_solver, or_solver

# Run and plot results
results = run_experiments()
# dqn_rewards, nn_rewards, or_rewards, ppo_rewards, maxent_rewards, sac_rewards, dqn_agent, maxent_agent, sac_agent, ppo_agent, nn_solver, or_solver
maxent_rewards, sac_rewards, maxent_agent, sac_agent = results

# Define methods to evaluate

methods = {
    #"DQN": lambda state: dqn_agent.select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
    "MaxEnt": lambda state: maxent_agent.select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
    "SAC": lambda state: sac_agent.select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
    #"PPO": ppo_agent,
    #"Nearest Neighbor": lambda env: nn_solver.solve(env.env),
    #"OR-Tools": lambda env: or_solver.solve(env.env)
}
# Compute total distance and evaluate methods
def get_route_from_method(env, method_name, method_fn):
    """Helper function to get route from different methods"""
    env.reset()
    current_pos = env.env.depot.copy()
    route = []

    try:
        if method_name in ["DQN", "MaxEnt", "SAC"]:
            # For RL agents
            state = env.reset()
            done = False
            while not done and len(route) < env.env.num_customers:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0)
                    action = method_fn(state_tensor)
                    if isinstance(action, torch.Tensor):
                        action = action.item()
                next_state, _, done, _ = env.step(action)
                route.append(action)
                state = next_state
        elif method_name == "PPO":
            # For PPO
            state = env.reset()
            done = False
            while not done and len(route) < env.env.num_customers:
                action, _ = method_fn.model.predict(state)
                next_state, _, done, _ = env.step(action)
                route.append(int(action))
                state = next_state
        else:
            # For OR-Tools and Nearest Neighbor
            route = method_fn(env)

    except Exception as e:
        print(f"Error in {method_name}: {str(e)}")
        return None

    return route

import time

# def compute_route_distance(env, route):
#     """Compute total distance for a given route"""
#     if route is None:
#         return float('inf')

#     env.reset()
#     total_distance = 0
#     current_pos = env.env.depot.copy()

#     for action in route:
#         try:
#             if action < 0 or action >= env.env.num_customers or env.env.visited[action]:
#                 continue

#             next_pos = env.env.customers[action]
#             total_distance += np.linalg.norm(current_pos - next_pos)
#             current_pos = next_pos
#             env.env.visited[action] = True

#         except Exception as e:
#             print(f"Error computing distance for action {action}: {str(e)}")
#             return float('inf')

#     # Add return to depot
#     total_distance += np.linalg.norm(current_pos - env.env.depot)
#     return total_distance

# # def evaluate_distances(env, methods, episodes=10):
# #     """Evaluate methods over multiple episodes"""
# #     all_distances = {method: [] for method in methods.keys()}

# #     for episode in range(episodes):
# #         for method_name, method_fn in methods.items():
# #             route = get_route_from_method(env, method_name, method_fn)
# #             distance = compute_route_distance(env, route)
# #             all_distances[method_name].append(distance)

# #     # Compute averages
# #     avg_distances = {method: np.mean(distances) for method, distances in all_distances.items()}
# #     return avg_distances

# def compute_route_metrics(env, route):
#     """Compute both distance and constraint violations for a route"""
#     if route is None:
#         return {'distance': float('inf'), 'time_violations': 0, 'capacity_violations': 0}

#     env.reset()
#     total_distance = 0
#     current_time = 0
#     current_load = 0
#     current_pos = env.env.depot.copy()
#     time_window_violations = 0
#     capacity_violations = 0

#     for action in route:
#         try:
#             if action < 0 or action >= env.env.num_customers:
#                 continue

#             # Calculate distance and time
#             next_pos = env.env.customers[action]
#             travel_time = np.linalg.norm(current_pos - next_pos)
#             total_distance += travel_time

#             # Update time
#             current_time += travel_time

#             # Check time window violations
#             if current_time > env.env.due_times[action]:
#                 time_window_violations += 1
#             elif current_time < env.env.ready_times[action]:
#                 current_time = env.env.ready_times[action]

#             # Add service time
#             current_time += env.env.service_times[action]

#             # Check capacity violations
#             current_load += env.env.demands[action]
#             if current_load > env.env.vehicle_capacity:
#                 capacity_violations += 1

#             current_pos = next_pos

#         except Exception as e:
#             print(f"Error computing metrics for action {action}: {str(e)}")
#             return {'distance': float('inf'), 'time_violations': 0, 'capacity_violations': 0}

#     # Return to depot
#     total_distance += np.linalg.norm(current_pos - env.env.depot)

#     return {
#         'distance': total_distance,
#         'time_violations': time_window_violations,
#         'capacity_violations': capacity_violations
#     }

# def get_route_from_method(env, method_name, method_fn):
#     """Helper function to get route from different methods"""
#     env.reset()
#     route = []
#     visited_customers = set()

#     try:
#         if method_name in ["DQN", "MaxEnt", "SAC"]:
#             # For RL agents
#             state = env.reset()
#             done = False
#             steps = 0
#             max_steps = env.env.num_customers * 2  # Allow some buffer for mistakes

#             while not done and steps < max_steps:
#                 steps += 1
#                 with torch.no_grad():
#                     state_tensor = torch.FloatTensor(state).unsqueeze(0)
#                     action = method_fn(state_tensor)
#                     if isinstance(action, torch.Tensor):
#                         action = action.item()

#                 # Check if action is valid
#                 if action not in visited_customers and action < env.env.num_customers:
#                     route.append(action)
#                     visited_customers.add(action)

#                 next_state, _, done, _ = env.step(action)
#                 state = next_state

#                 # Check if all customers have been visited
#                 if len(visited_customers) == env.env.num_customers:
#                     done = True

#         elif method_name == "PPO":
#             # For PPO
#             state = env.reset()
#             done = False
#             steps = 0
#             max_steps = env.env.num_customers * 2

#             while not done and steps < max_steps:
#                 steps += 1
#                 action, _ = method_fn.model.predict(state)

#                 # Check if action is valid
#                 if action not in visited_customers and action < env.env.num_customers:
#                     route.append(int(action))
#                     visited_customers.add(action)

#                 next_state, _, done, _ = env.step(action)
#                 state = next_state

#                 # Check if all customers have been visited
#                 if len(visited_customers) == env.env.num_customers:
#                     done = True

#         else:
#             # For OR-Tools and Nearest Neighbor
#             route = method_fn(env)
#             visited_customers = set(route)

#         # Validate route
#         if len(visited_customers) != env.env.num_customers:
#             print(f"Warning: {method_name} did not visit all customers. Visited {len(visited_customers)}/{env.env.num_customers}")
#             return None

#     except Exception as e:
#         print(f"Error in {method_name}: {str(e)}")
#         return None

#     return route

def compute_route_metrics(env, route):
    """Compute both distance and constraint violations for a route"""
    if route is None:
        print("Route is None")
        return {'distance': float('inf'), 'time_violations': 0, 'capacity_violations': 0}

    env.reset()
    total_distance = 0
    current_time = 0
    current_load = 0
    current_pos = env.env.depot.copy()
    time_window_violations = 0
    capacity_violations = 0
    visited = set()

    # Check if all customers were visited
    if len(visited) != env.env.num_customers:
        return {'distance': float('inf'), 'time_violations': env.env.num_customers, 'capacity_violations': env.env.num_customers}

    # Return to depot
    total_distance += np.abs(current_pos - env.env.depot).sum()

    # Check final time window violation for depot return
    if current_time + np.abs(current_pos - env.env.depot).sum() > max(env.env.due_times):
        time_window_violations += 1

    return {
        'distance': total_distance,
        'time_violations': time_window_violations,
        'capacity_violations': capacity_violations
    }

def evaluate_methods_fairly(env, methods, episodes=10):
    """Evaluate all methods using the same metrics"""
    results = {method: {
        'distances': [],
        'time_violations': [],
        'capacity_violations': [],
        'execution_times': []
    } for method in methods.keys()}

    for episode in range(episodes):
        for method_name, method_fn in methods.items():
            start_time = time.time()
            route = get_route_from_method(env, method_name, method_fn)
            execution_time = time.time() - start_time

            metrics = compute_route_metrics(env, route)
            print("Dist",metrics['distance'])
            results[method_name]['distances'].append(metrics['distance'])
            results[method_name]['time_violations'].append(metrics['time_violations'])
            results[method_name]['capacity_violations'].append(metrics['capacity_violations'])
            results[method_name]['execution_times'].append(execution_time)

    # Create summary DataFrame using list comprehension instead of append
    summary_data = [
        {
            'Method': method,
            'Avg Distance': np.mean(results[method]['distances']),
            'Avg Time Violations': np.mean(results[method]['time_violations']),
            'Avg Capacity Violations': np.mean(results[method]['capacity_violations']),
            'Avg Execution Time (s)': np.mean(results[method]['execution_times'])
        }
        for method in results
    ]

    return pd.DataFrame(summary_data)

# # Evaluate distances
# print("\nEvaluating Total Distances...")
# avg_distances = evaluate_distances(GymVRPEnvironment(num_customers=10), methods)

# # Generate a table of results
# results_table = pd.DataFrame({
#     "Method": list(avg_distances.keys()),
#     "Average Distance": list(avg_distances.values())
# })
# print("\nResults Table:")
# print(results_table)

# Evaluate distances and times
print("\nEvaluating Performance...")
summary = evaluate_methods_fairly(GymVRPEnvironment(num_customers=100), methods)

# Generate a table of results
# results_table = pd.DataFrame({
#     "Method": list(avg_distances.keys()),
#     "Average Distance": list(avg_distances.values()),
#     "Average Time (s)": list(avg_times.values())
# })
print("\nResults Table:")
print(summary)

# Save results to CSV for analysis
summary.to_csv("vrp_performance.csv", index=False)
print("Performance results saved to 'vrp_performance.csv'.")

# Plotting
plt.figure(figsize=(15, 5))

# Raw rewards
plt.subplot(1, 2, 1)
plt.plot(maxent_rewards, label='MaxEnt', alpha=0.6)
plt.plot(sac_rewards, label='SAC', alpha=0.6)
plt.plot(dqn_rewards, label='DQN', alpha=0.6)
plt.plot(ppo_rewards, label='PPO', alpha=0.6)
plt.title('Training Rewards')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.legend()

# Moving average
window = 20
plt.subplot(1, 2, 2)
plt.plot(np.convolve(maxent_rewards, np.ones(window)/window, mode='valid'),
         label='MaxEnt', alpha=0.8)
plt.plot(np.convolve(sac_rewards, np.ones(window)/window, mode='valid'),
         label='SAC', alpha=0.8)
plt.plot(np.convolve(dqn_rewards, np.ones(window)/window, mode='valid'),
         label='DQN', alpha=0.8)
plt.plot(np.convolve(ppo_rewards, np.ones(window)/window, mode='valid'),
         label='PPO', alpha=0.8)
plt.title(f'Moving Average ({window} episodes)')
plt.xlabel('Episode')
plt.ylabel('Average Reward')
plt.legend()

plt.tight_layout()
plt.show()

# Print final statistics
def print_stats(rewards, name):
    print(f"\n{name} Statistics:")
    print(f"Final Average (last 20 episodes): {np.mean(rewards[-20:]):.2f}")
    print(f"Best Episode: {max(rewards):.2f}")
    print(f"Worst Episode: {min(rewards):.2f}")
    print(f"Overall Average: {np.mean(rewards):.2f}")
# Keep all previous code the same until the plotting section, then replace with:

def plot_comparison(maxent_rewards, sac_rewards, dqn_rewards, ppo_rewards, window=10):
    plt.figure(figsize=(15, 5))

    # Get number of episodes
    episodes = len(maxent_rewards)
    x_axis = np.arange(episodes)

    # Raw rewards
    plt.subplot(1, 2, 1)
    plt.plot(x_axis, maxent_rewards, label='MaxEnt', alpha=0.6)
    plt.plot(x_axis, sac_rewards, label='SAC', alpha=0.6)
    plt.plot(x_axis, dqn_rewards, label='DQN', alpha=0.6)
    plt.plot(x_axis, ppo_rewards, label='PPO', alpha=0.6)
    plt.title('Training Rewards (Raw)')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Moving average
    def get_moving_average(rewards, window):
        weights = np.ones(window) / window
        return np.convolve(rewards, weights, mode='valid')

    # Calculate moving averages
    ma_maxent = get_moving_average(maxent_rewards, window)
    ma_sac = get_moving_average(sac_rewards, window)
    ma_dqn = get_moving_average(dqn_rewards, window)
    ma_ppo = get_moving_average(ppo_rewards, window)

    # Adjust x-axis for moving average plot to align with original episodes
    ma_x = np.arange(window-1, episodes)

    plt.subplot(1, 2, 2)
    plt.plot(ma_x, ma_maxent, label='MaxEnt', alpha=0.8)
    plt.plot(ma_x, ma_sac, label='SAC', alpha=0.8)
    plt.plot(ma_x, ma_dqn, label='DQN', alpha=0.8)
    plt.plot(ma_x, ma_ppo, label='PPO', alpha=0.8)
    plt.title(f'Training Rewards ({window}-Episode Moving Average)')
    plt.xlabel('Episode')
    plt.ylabel('Average Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# After training, call the plotting function with a smaller window
print("Plotting results...")
plot_comparison(maxent_rewards, sac_rewards, dqn_rewards, ppo_rewards, window=10)  # Changed to 10-episode window

# Print statistics with both raw and smoothed metrics
def print_stats(rewards, name, window=10):
    print(f"\n{name} Statistics:")
    print(f"Raw Metrics:")
    print(f"Final Average (last {window} episodes): {np.mean(rewards[-window:]):.2f}")
    print(f"Best Episode: {max(rewards):.2f}")
    print(f"Worst Episode: {min(rewards):.2f}")
    print(f"Overall Average: {np.mean(rewards):.2f}")
    def get_moving_average(rewards, window):
        weights = np.ones(window) / window
        return np.convolve(rewards, weights, mode='valid')
    # Smoothed metrics
    smoothed = get_moving_average(rewards, window)
    print(f"\nSmoothed Metrics ({window}-episode window):")
    print(f"Best Smoothed Performance: {max(smoothed):.2f}")
    print(f"Final Smoothed Performance: {smoothed[-1]:.2f}")

print("\nPerformance Statistics:")
print_stats(maxent_rewards, "MaxEnt")
print_stats(sac_rewards, "SAC")
print_stats(dqn_rewards, "DQN")

# Bar chart for solution quality comparison
methods = ["DQN", "Nearest Neighbor", "OR-Tools", "PPO"]
average_rewards = [np.mean(dqn_rewards), np.mean(nn_rewards), np.mean(or_rewards), np.mean(ppo_rewards)]

plt.figure(figsize=(8, 5))
plt.bar(methods, average_rewards, color=['blue', 'orange', 'green', 'purple'])
plt.xlabel("Method")
plt.ylabel("Average Reward (Solution Quality)")
plt.title("Solution Quality Comparison")
plt.show()

test_env = GymVRPEnvironment(num_customers=10)

print("\nEvaluating Total Distances...")
avg_distances = evaluate_distances(test_env, methods)

print("\nDistances Table:")
distances_df = pd.DataFrame({
    "Method": list(avg_distances.keys()),
    "Average Distance": list(avg_distances.values())
})
print(distances_df)

print("\nEvaluating Performance...")
avg_distances, avg_times = evaluate_performance(test_env, methods)

print("\nPerformance Table:")
performance_df = pd.DataFrame({
    "Method": list(avg_distances.keys()),
    "Average Distance": list(avg_distances.values()),
    "Average Time (s)": list(avg_times.values())
})
print(performance_df)

def visualize_routes(env, methods):
    plt.figure(figsize=(12, 8))
    depot = env.env.depot
    customers = env.env.customers

    for method, action_fn in methods.items():
        env.reset()
        actions = action_fn(env)
        route = [depot] + [customers[a] for a in actions if isinstance(a, int) and 0 <= a < len(customers)] + [depot]
        route = np.array(route)

        plt.plot(route[:, 0], route[:, 1], marker='o', label=method)
        plt.scatter(depot[0], depot[1], c='red', s=100, label='Depot')

    plt.title("Routes Generated by Different Methods")
    plt.xlabel("X-coordinate")
    plt.ylabel("Y-coordinate")
    plt.legend()
    plt.grid(True)
    plt.show()

visualize_routes(GymVRPEnvironment(num_customers=100), methods)

# Save results to CSV
data = {
    "DQN Rewards": dqn_rewards,
    "Nearest Neighbor Rewards": nn_rewards,
    "OR-Tools Rewards": or_rewards,
    "PPO Rewards": ppo_rewards
}
df = pd.DataFrame(data)
df.to_csv("vrp_results.csv", index=False)
print("Results saved to 'vrp_results.csv'.")

Test on Solomon data

In [ ]:
def load_solomon_c101(file_path):
    """Load and process Solomon C101 dataset"""
    try:
        solomon_data = parse_solomon_dataset(file_path)

        # Create environment with Solomon data
        env = GymVRPEnvironment(solomon_data=solomon_data)
        return env

    except Exception as e:
        print(f"Error loading Solomon dataset: {str(e)}")
        return None

def run_solomon_experiments(file_path, episodes=1000):
    """Run experiments using Solomon dataset"""
    # Load environment with Solomon data
    env = load_solomon_c101(file_path)
    if env is None:
        return None

    # Get state and action dimensions from environment
    state = env.reset()
    state_dim = len(state)
    action_dim = env.action_space.n

    print(f"Solomon Environment initialized with:")
    print(f"- Number of customers: {env.env.num_customers}")
    print(f"- State dimension: {state_dim}")
    print(f"- Action dimension: {action_dim}")

    # Initialize agents
    maxent_agent = MaxEntRL(state_dim, action_dim, hidden_dim=256)
    sac_agent = SACAgent(state_dim, action_dim, hidden_dim=256)
    dqn_agent = DQNAgent(state_dim, action_dim, hidden_dim=256)
    nn_solver = NearestNeighbor()
    ppo_env = load_solomon_c101(file_path)  # Create separate env for PPO
    ppo_agent = PPOAgent(
        ppo_env,
        verbose=0,
        policy_kwargs={'net_arch': [256, 256]}
    )
    or_solver = ORToolsSolver()

    # History tracking
    or_rewards = []
    ppo_rewards = []
    maxent_rewards = []
    sac_rewards = []
    dqn_rewards = []
    nn_rewards = []

    # Train PPO Agent
    print("\nTraining PPO...")
    ppo_agent.train(timesteps=episodes * 100)

    # Training loop for other RL agents
    for algorithm, agent, rewards in [
        ("MaxEnt", maxent_agent, maxent_rewards),
        ("SAC", sac_agent, sac_rewards),
        ("DQN", dqn_agent, dqn_rewards),
    ]:
        print(f"\nTraining {algorithm}...")

        for episode in tqdm(range(episodes)):
            state = env.reset()
            episode_reward = 0
            steps = 0
            done = False

            while not done and steps < env.env.num_customers * 2:
                steps += 1
                action = agent.select_action(state)
                next_state, reward, done, _ = env.step(action)

                agent.store_transition(state, action, reward, next_state, done)
                agent.update()

                episode_reward += reward
                state = next_state

            rewards.append(episode_reward)

            if (episode + 1) % 20 == 0:
                mean_reward = np.mean(rewards[-20:])
                print(f"\n{algorithm} Episode {episode+1}, Average Reward: {mean_reward:.2f}")

    # Evaluate heuristic methods
    print("\nRunning Nearest Neighbor...")
    for _ in tqdm(range(episodes)):
        route = nn_solver.solve(env.env)
        metrics = compute_route_metrics(env, route)
        nn_rewards.append(-metrics['distance'])  # Convert distance to reward

    print("\nRunning OR-Tools Solver...")
    for _ in tqdm(range(episodes)):
        route = or_solver.solve(env.env)
        metrics = compute_route_metrics(env, route)
        or_rewards.append(-metrics['distance'])  # Convert distance to reward

    # Evaluate PPO
    print("\nEvaluating PPO...")
    ppo_rewards = ppo_agent.evaluate(episodes=episodes)

    return {
        'env': env,
        'rewards': {
            'DQN': dqn_rewards,
            'MaxEnt': maxent_rewards,
            'SAC': sac_rewards,
            'PPO': ppo_rewards,
            'Nearest Neighbor': nn_rewards,
            'OR-Tools': or_rewards
        },
        'agents': {
            'DQN': dqn_agent,
            'MaxEnt': maxent_agent,
            'SAC': sac_agent,
            'PPO': ppo_agent,
            'Nearest Neighbor': nn_solver,
            'OR-Tools': or_solver
        }
    }

# Usage example:
file_path = '/content/c101.txt'
results = run_solomon_experiments(file_path, episodes=1000)

if results:
    env = results['env']
    rewards = results['rewards']
    agents = results['agents']

    # Evaluate methods using fair comparison
    methods = {
        "DQN": lambda state: agents['DQN'].select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
        "MaxEnt": lambda state: agents['MaxEnt'].select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
        "SAC": lambda state: agents['SAC'].select_action(state.numpy() if isinstance(state, torch.Tensor) else state),
        "PPO": agents['PPO'],
        "Nearest Neighbor": lambda env: agents['Nearest Neighbor'].solve(env.env),
        "OR-Tools": lambda env: agents['OR-Tools'].solve(env.env)
    }

    # Evaluate performance
    print("\nEvaluating Performance...")
    summary = evaluate_methods_fairly(env, methods)
    print("\nResults Summary:")
    print(summary)

    # Plot results
    plot_comparison(
        rewards['MaxEnt'],
        rewards['SAC'],
        rewards['DQN'],
        rewards['PPO'],
        window=10
    )

    # Visualize routes
    visualize_routes(env, methods)